In [1]:
# Data wrangling
import numpy as np
import pandas as pd  # Not a requirement of giotto-tda, but is compatible with the gtda.mapper module

# Data viz
import plotly.graph_objects as go
import plotly.express as px

from ipywidgets import HTML, HBox, VBox, Image, Layout

# TDA magic
from gtda.mapper import (
    CubicalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph,
)
from gtda.mapper.utils.visualization import set_node_sizeref

# ML tools
from sklearn import datasets
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
data, _ = datasets.make_circles(n_samples=5000, noise=0.05, factor=0.3, random_state=42)

fig= go.FigureWidget(
    data=[
        dict
            (
            type='scatter',
            x=data[:, 0], 
            y=data[:, 1], 
            mode="markers",
            )
        ],
)
fig.layout.hovermode = 'closest'
fig

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '3074a5b…

In [3]:
# Define filter function - can be any scikit-learn transformer
filter_func = Projection(columns=[0, 1])
# Define cover
cover = CubicalCover(n_intervals=10, overlap_frac=0.3)
# Choose clustering algorithm - default is DBSCAN
clusterer = DBSCAN()

# Configure parallelism of clustering step
n_jobs = 1

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=n_jobs,
)

In [4]:
fig_mapper = plot_static_mapper_graph(pipe, data)
# Display figure
#fig_mapper.show(config={"scrollZoom": True})

In [5]:
img_rgb = np.array([[[255, 0, 0], [0, 255, 0], [0, 0, 255]],
                    [[0, 255, 0], [0, 0, 255], [255, 0, 0]],
                    [[0, 255, 255], [255, 0, 255], [0, 255, 0]]
                   ], dtype=np.uint8)
fig_matrix = go.FigureWidget(go.Image(z=img_rgb))
# fig_matrix.show()

In [6]:
scatter = fig_mapper.data[1]
scatter
#print(len(scatter_mapper))

Scatter({
    'hoverinfo': 'text',
    'hovertext': [Node ID:0<br>Node size:4, Node ID:1<br>Node size:60, Node
                  ID:2<br>Node size:122, Node ID:3<br>Node size:130, Node
                  ID:4<br>Node size:129, Node ID:5<br>Node size:125, Node
                  ID:6<br>Node size:88, Node ID:7<br>Node size:8, Node ID:8<br>Node
                  size:6, Node ID:9<br>Node size:91, Node ID:10<br>Node size:132,
                  Node ID:11<br>Node size:58, Node ID:12<br>Node size:14, Node
                  ID:13<br>Node size:11, Node ID:14<br>Node size:50, Node
                  ID:15<br>Node size:122, Node ID:16<br>Node size:118, Node
                  ID:17<br>Node size:8, Node ID:18<br>Node size:65, Node
                  ID:19<br>Node size:136, Node ID:20<br>Node size:15, Node
                  ID:21<br>Node size:1, Node ID:22<br>Node size:4, Node
                  ID:23<br>Node size:1, Node ID:24<br>Node size:7, Node
                  ID:25<br>Node size:134, Node ID:26<b

In [7]:
graph = pipe.fit_transform(data)
graph["node_metadata"].keys()

dict_keys(['node_id', 'pullback_set_label', 'partial_cluster_label', 'node_elements'])

In [8]:
node_id, node_elements = (graph["node_metadata"]["node_id"],
                          graph["node_metadata"]["node_elements"],
                          )

print("Node Id: {}, \nNode elements: {}, \nData points: {}".format(node_id[0], 
                                                                 node_elements[0], 
                                                                 data[node_elements[0]]
                                                                 )
)

Node Id: 0, 
Node elements: [1675 1944 2425 4464], 
Data points: [[-0.85719291 -0.74172245]
 [-0.88187702 -0.69106254]
 [-0.8881879  -0.63188784]
 [-0.84665374 -0.73453227]]


In [9]:
indices = node_elements[0]
indices

x_mean = np.mean(data[indices], axis=0)
x_mean[0]

-0.8684778940159361

In [23]:
node_stats = HTML()

# Config for to_html() is Pandas, not Plotly or ipywidgets
pandas_config={'titleText': True}


def hover_node_stats(trace, points, state):
    ind = points.point_inds
    indices = node_elements[int(ind[0])]
    values = pd.DataFrame(data[indices])
    values.columns = ['x', 'y']
    x_mean = values.mean()
    x_mean.name = 'Mean values of features'
    node_stats.value = x_mean.to_frame().to_html(header=True)

    
scatter.on_hover(hover_node_stats)

In [24]:
v_box_layout = Layout(display='flex',
                      flex_flow='row',
                      align_items='center',
                      justify_content='center',
                      width='80%')



In [25]:
dashboard = VBox([fig_mapper, 
                  HBox([node_stats, fig_matrix], layout=v_box_layout)
                 ])
dashboard

    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'widt…